In [1]:
# Note: Although the file is named '2024_filtered_bus_data.csv', it contains cleaned data ranging from 2018 to 2024, with 2024 being the most recent year.

import pandas as pd

df_2017_2018 = pd.read_csv('cleaned_data/2017_2018_bus_only.csv')
df_2024 = pd.read_csv('cleaned_data/2024_filtered_bus_data.csv')

In [2]:
df_2017_2018.head()
df_2024.head()

,month_of_service,daytype,daycount,route_or_line,ridership_total,ridership_average,ObjectId
0,2024-12-01 05:00:00+00:00,Total,31,Silver Line,836989,27000,47
1,2024-07-01 04:00:00+00:00,Total,31,Silver Line,880384,28399,57
2,2024-11-01 04:00:00+00:00,Total,30,Silver Line,946230,31541,149
3,2024-08-01 04:00:00+00:00,Total,31,Silver Line,870265,28073,155
4,2024-10-01 04:00:00+00:00,Total,31,Silver Line,950838,30672,201


In [4]:
# Standardize 2017-2018 data

df_2017_2018['date'] = pd.to_datetime(df_2017_2018['service_date'])
df_2017_2018['year'] = df_2017_2018['date'].dt.year
df_2017_2018['ridership_total'] = df_2017_2018['total_monthly_ridership']
df_2017_2018['ridership_average'] = df_2017_2018['average_monthly_ridership']

df_2017_2018_cleaned = df_2017_2018[['year', 'date', 'route_or_line', 'ridership_total', 'ridership_average']]

# Standardize 2018-2024 data 

df_2024['date'] = pd.to_datetime(df_2024['month_of_service'])
df_2024['year'] = df_2024['date'].dt.year

# Clean the 2018-2024 data, where daytype == 'Total' only 
df_2024 = df_2024[df_2024['daytype'] == 'Total'].copy()

df_2024['date'] = pd.to_datetime(df_2024['month_of_service'])
df_2024['year'] = df_2024['date'].dt.year

df_2024['ridership_total'] = df_2024['ridership_total']
df_2024['ridership_average'] = df_2024['ridership_average']

df_2024_cleaned = df_2024[['year', 'date', 'route_or_line', 'ridership_total', 'ridership_average']]

# Combine both datasets
df_combined = pd.concat([df_2017_2018_cleaned, df_2024_cleaned], ignore_index=True)

# Sort first by year, then route (Bus, Silver Line), then date 
route_order = ['Bus', 'Silver Line']
df_combined['route_order'] = df_combined['route_or_line'].apply(lambda x: route_order.index(x) if x in route_order else len(route_order))
df_combined = df_combined.sort_values(by=['year', 'route_order', 'date']).drop(columns=['route_order'])



In [5]:
df_combined.head()

,year,date,route_or_line,ridership_total,ridership_average
10,2017,2017-01-01 05:00:00+00:00,Bus,8003644.0,242534.0
17,2017,2017-02-01 05:00:00+00:00,Bus,7301412.0,260764.0
16,2017,2017-03-01 05:00:00+00:00,Bus,8595017.0,277259.0
0,2017,2017-04-01 04:00:00+00:00,Bus,8098478.0,269949.0
1,2017,2017-05-01 04:00:00+00:00,Bus,8817889.0,284448.0


In [ ]:
df_combined.rename(columns={"date": "month_of_service"}, inplace=True)

In [6]:
df_combined.to_csv('combined_bus_silverline_2017_2024.csv', index=False)